In [1]:
import numpy as np
import re
import random
import transformers
from transformers import GPT2Tokenizer
import string
import torch
from torch.optim import AdamW
import textwrap

/Users/ivanyuminov/ds_bootcamp/ds_offline/virtual_env/pytorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
# !pip install transformers

In [2]:
# открываем файл
with open('/Users/ivanyuminov/ds_bootcamp/NLP_project/text/text_VV.txt', encoding='utf8') as f:
    text = f.read()
text_VV = re.sub('\n{2,}', '\n', text)

In [3]:
def clean(text):
    text = re.sub(r'\d+', ' ', text) # удаляем числа
    text = text.translate(str.maketrans('', '', '"#$%&\'()*+-/:;<=>@[\\]^_`{|}~')) # удаляем знаки пунктуации
    return text

text_VV = clean(text_VV)

In [4]:
# Токенайзер модели для данных 
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
tokens = tokenizer.encode(text_VV, add_special_tokens=True)
tokens = np.array(tokens)

In [5]:
# разбиваем на train и test
l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)
print(len(tokens), len(train), len(test))

48326 38652 9663


In [6]:
from transformers import GPT2LMHeadModel

# Эту модель просто подгружаем и не будем дообучать 
model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)


# Эту модель подгрузим и далее обучим
model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False)

In [7]:
batch_size = 8
max_len = 256
epochs = 7

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
# print(n_train, n_test)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

# трансформеры с трудом обучаются, для них нужны разные способы повышения 
# эффективности градиентного спуска
total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

In [8]:
# готовим тензоры для обучения
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids)
    return batch_ids
    
# обучающий цикл
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    # pbar = tqdm(range(n_train))
    # for i in pbar:
    for i in range(n_train):
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None, 
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        # pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    # pbar = tqdm(range(n_test))
    # for i in pbar:
    for i in range(n_test):
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        # pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)

epoch 1/7 : training
epoch 1/7 : validation
epoch 2/7 : training
epoch 2/7 : validation
epoch 3/7 : training
epoch 3/7 : validation
epoch 4/7 : training
epoch 4/7 : validation
epoch 5/7 : training
epoch 5/7 : validation
epoch 6/7 : training
epoch 6/7 : validation
epoch 7/7 : training
epoch 7/7 : validation


In [9]:
torch.save(model.state_dict(), 'weights_of_preprocessing_text.pt')

In [10]:
# модель без дообучения
prompt = 'Ничего нет кругом кроме пустоты и тумана'
prompt = tokenizer.encode(prompt, return_tensors='pt')
out = model_init.generate(
    input_ids=prompt,
    max_length=25,
    num_beams=5,
    do_sample=True,
    temperature=100.,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=3,
    num_return_sequences=7,
    ).numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ничего нет кругом кроме пустоты и тумана, который так не может удержать никого кроме своей цели. Только здесь - и я
чувствую,
------------------
Ничего нет кругом кроме пустоты и тумана и что не будет никого кроме этого всего! Таких много на нашем маленьком
планетном
------------------
Ничего нет кругом кроме пустоты и тумана - все покрылось пылью - там все так странно и одиноко А это - ни с
------------------
Ничего нет кругом кроме пустоты и тумана... И что ты видишь в том кроме этого тумана Что это значит Что есть пустота?
------------------
Ничего нет кругом кроме пустоты и тумана..  Все что было когда либо живым... только и всего.   Не могу сказать
------------------
Ничего нет кругом кроме пустоты и тумана, а вокруг тишина - так я могу стоять один? Мне кажется это все очень глупо.
------------------
Ничего нет кругом кроме пустоты и тумана - всё в мире как одна огромная река - ничего вокруг как всегда нету Есть
пустота...
------------------


In [11]:
# дообученная модель
prompt = 'Ничего нет кругом кроме пустоты и тумана'
prompt = tokenizer.encode(prompt, return_tensors='pt')
out = model.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=5,
    do_sample=True,
    temperature=1,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=2,
    num_return_sequences=1).numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 100), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ничего нет кругом кроме пустоты и тумана.  И нет в мире ничего, что не было бы пустотой.   Я знаю,
ты не поверишь, но я не верю,  Что в этом мире есть пустота и туман,   Что там нет ни души, ни
света, и нет никого, кто мог бы сказать мне что-то, кроме тебя, мой друг, моя любовь, мое счастье,
мои слезы, твои поцелуи, твой голос, твоя улыбка, твое дыхание, наше дыхание...  Я не могу поверить
в то, о чем ты так долго думал, не может быть, чтобы все было так, как ты хочешь, а не так как я
хочу, потому что я люблю тебя и хочу быть с тобой всегда, пока ты жив
------------------
